# Predict Future Sales using SARIMA

Kaggle Competition: https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview

<br>

**Table of Contents:**

* [Import libraries & define functions](#section-1)
* [Import data](#section-2)
* [EDA (Exploratory Data Analysis)](#section-3)
    - [Data Overview](#section-3-1)
    - [Data Summarization](#section-3-2)
    - [Missing Value](#section-3-3)
    - [Overview of items count per month](#section-3-4)
    - [Seasonal or Non-seasonal?](#section-3-5)
    - [Trend or Non-trend?](#section-3-6)
    - [Non-stationary or stationary?](#section-3-7)
* [Data Cleaning](#section-4)
* [SARIMA](#section-5)
    - [Build S-ARIMA model on total sales](#section-5-1)
    - [Submission](#section-5-2)

<a id="section-1"></a>
# Import libraries & define functions

In [ ]:
# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Time-series manipulation
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pandas import Series

# ARIMA
!pip install pmdarima > /dev/null
import pmdarima as pm

In [ ]:
# Stationarity test - ADF
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

# Create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# # Invert the differenced forecast
# def inverse_difference(last_ob, value):
#     return value + last_ob

<a id="section-2"></a>
# Import data

Data from the competition: https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales/data

In [ ]:
# Import data
sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")

# Date re-format
sales['date']=sales['date'].apply(
    lambda x:datetime.datetime.strptime(x, '%d.%m.%Y')
)

sales

In [ ]:
# Import data
pairs = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

pairs

<a id="section-3"></a>
# EDA (Exploratory Data Analysis)

<a id="section-3-1"></a>
## Data Overview 

In [ ]:
sales.shape

In [ ]:
sales.info()

|Feature|Data type|     Meaning  |
|:--------:|:-------------:|:-----|
|date | datetime|date in format dd/mm/yyyy|
|date_block_num |int| a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,...,|
|shop_id | int | unique identifier of a shop|
|item_id | int |unique identifier of a product|
|item_price | float |current price of an item|
|item_cnt_day |float| number of products sold|




|Đặc trưng|Kiểu dữ liệu|Ý nghĩa|
|:--------:|:-------------:|:-----|
|date | datetime|Ngày tháng năm theo định dạng dd/mm/yyyy|
|date_block_num |int| Số tháng liên tiếp kể từ tháng 1 năm 2013. Tháng 1 năm 2013 là 0, tháng 2 năm 2013 là 1,...|
|shop_id | int | mã định danh của shop|
|item_id | int |mã định danh của item|
|item_price | float |giá hiện tại của item|
|item_cnt_day |float| số lượng sản phẩm đã bán|

<a id="section-3-2"></a>
## Data Summarization
We analyze numerical data since practically all features have a numerical type without a feature `date`.

In [ ]:
num_sales = sales.select_dtypes(include=['int64','float64'])
num_sales.describe()

<a id="section-3-3"></a>
## Missing Value

In [ ]:
def missing_ratio(s):
    return s.isna().mean()*100
def nums_diff_values(s):
    return s.dropna().nunique()
def diff_vals_ratio(s):
    s = s.dropna()
    return (s.value_counts()/len(s)*100).to_dict()

num_sales.agg([missing_ratio, nums_diff_values, diff_vals_ratio])

We assure that there are no missing values in dataset.

In [ ]:
num_sales.hist(figsize=(10,10),bins=200);

**Comments:**
- The number of date  consecutive month (`date_num_block`) have a relatively uniform distribution, indicating that there are consistent transactions. The seventh and twenty-third months (from January 2013) have many transactions which is higher than the others. 
- Shop_id reach the highest transaction is 31. Other shops conduct transaction that are less than 150.000
- The number of transaction's items reach the highest is 60.000. Other shops conduct transaction that are less than 45.000
- Item prices (`item_price`) are often less than 25.000, with only a few things over 300.000.


In [ ]:
sns.heatmap(num_sales.corr(), annot=True)
plt.show()

<a id="section-3-4"></a>
## Overview of items count per month

In [ ]:
sales.copy().set_index('date').item_cnt_day.resample('M').sum().plot()
plt.show()

In [ ]:
sales.copy().set_index('date').item_cnt_day.resample('M').mean().plot()
plt.show()

**Comments:**
- The average items count of September 2015 reach the highest value.
- All values follow pattern by half a year, which means that after a half-year, the item count value will fall and then rise again after a few months. There are generally two peaks in a year.


<a id="section-3-5"></a>
## Seasonal or Non-seasonal?

In [ ]:
total_sales = sales \
    .groupby(["date_block_num"])["item_cnt_day"] \
    .sum() \
    .astype('float')

plt.title('Total Sales of the company')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.plot(total_sales);

**Comments:**
* We will treat the data as **seasonal data**, because there are peak sales per 12-months-period

<a id="section-3-6"></a>
## Trend or Non-trend?

In [ ]:
plt.plot(total_sales.rolling(window=12,center=False).mean(),label='Rolling Mean');
plt.plot(total_sales.rolling(window=12,center=False).std(),label='Rolling Standard Deviation');
plt.legend();

**Comments:**
* We will treat the data as it has **decrease-trend**, because the rolling mean decreases over time.

<a id="section-3-7"></a>
## Non-stationary or stationary?

In [ ]:
res = sm.tsa.seasonal_decompose(total_sales.values, period = 12, model="additive") \
        .plot()

In [ ]:
test_stationarity(total_sales)

**Comments:**
* We will treat the data as **non-stationary data**, because the p-value is greater than 5% (and as we observe in the plot)

<a id="section-4"></a>
# Data Cleaning

In [ ]:
#trucquynh

# Merge some duplicate shops
sales["shop_id"] = sales["shop_id"].replace({0: 57, 1: 58, 11: 10, 40: 39})
# Keep only shops that are in the test set
sales = sales.loc[sales.shop_id.isin(pairs["shop_id"].unique()), :]
# Drop training items with extreme or negative prices or sales counts
sales = sales[(sales["item_price"] > 0) & (sales["item_price"] < 50000)]
sales = sales[(sales["item_cnt_day"] > 0) & (sales["item_cnt_day"] < 1000)]

<a id="section-5"></a>
# SARIMA

Mô hình $SARIMA$ (Seasonal AutoRegressive Integrated Moving Average) cấu hình từ 7 tham số:

$$SARIMA(p,d,q)(P,D,Q)[m]$$

Với:

3 tham số $p,d,q$ tương tự như ở mô hình $ARIMA$:

* $p$: Bậc của mô hình $AR$ - autoregressive (số lượng độ trễ thời gian - "lags" của dữ liệu dùng để dự đoán).

* $d$: Mức độ chênh lệch - regular differencing (số lần mà dữ liệu trừ đi các giá trị trong quá khứ) cần thiết để khiến cho dữ liệu tĩnh (stationary, not trendy). Nếu dữ liệu đầu vào đã tĩnh (stationary) sẵn thì $d=0$.

* $q$: Bậc của mô hình $MA$ - moving-average (số lượng lỗi - lagged forecast error được đưa vào mô hình).

Mô hình $SARIMA$ bổ sung thêm 3 tham số bắt buộc và 1 tham số tuỳ chọn, có liên quan đến tính chu kỳ (seasonal) như sau:

* $P$: Tương tự $p$, nhưng mà là bậc của mô hình $SAR$ - seasonal autoregressive.

* $D$: Tương tự $d$, nhưng là mức độ chênh lệch theo chu kỳ - seasonal differencing.

* $Q$: Tương tự $q$, nhưng mà là bậc của mô hình $SMA$ - seasonal moving-average.

* $m$: Số bước/giai đoạn của một chu kỳ thời gian (Ví dụ: Chu kỳ thời gian là 1 năm, chúng ta có dữ liệu của mỗi tháng, vậy $m=12$ vì 1 năm có 12 tháng)

<a id="section-5-1"></a>
## Build S-ARIMA model on total sales

In [ ]:
data_df = sales \
    .groupby(['date_block_num']) \
    ['date', 'item_cnt_day'] \
    .agg({'date':'min', 'item_cnt_day':'sum'})

data_df.rename(columns={"item_cnt_day": "item_cnt_month"}, inplace=True)
data_df.set_index(['date'], inplace=True)

data_df.head()

In [ ]:
plt.plot(data_df)
plt.tight_layout()
plt.show()

In [ ]:
n = len(data_df)
train_test_ratio = 1.0 # 0.8
train_sr = data_df['item_cnt_month'][:int(n*train_test_ratio)]
test_sr = data_df['item_cnt_month'][int(n*train_test_ratio):]

print(f"Train/Test: {len(train_sr)}/{len(test_sr)}")

In [ ]:
sarima_model = pm.auto_arima(
    y=train_sr,                # Dữ liệu train
    stationary=False,          # Dữ liệu không tĩnh mà có tính xu hướng - Trend
    seasonal=True,             # Dữ liệu có tính cho kỳ - Seasonal
    test='kpss',               # Test: Kwiatkowski–Phillips–Schmidt–Shin
    seasonal_test ='ocsb',     # Test: Osborn-Chui-Smith-Birchenhall
    start_p=1, max_p=4,        # Tìm p trong khoảng [1,4]
    d=None,                    # Tìm d dựa trên tham số test (mặc định test='kpss')
    start_q=1, max_q=4,        # Tìm q trong khoảng [1,4]
    start_P=1, max_P=4,        # Tìm P trong khoảng [1,4]
    D=None,                    # Tìm D dựa trên tham số seasonal_test (mặc định test='ocsb')
    start_Q=1, max_Q=4,        # Tìm Q trong khoảng [1,4]
    m=12,                      # m=12 vì dữ liệu có tính chu kỳ 12 tháng
    trace=True,
    stepwise=True,
)

In [ ]:
print(sarima_model.summary())

In [ ]:
predicted_values, confint = sarima_model.predict(
    n_periods=3*12, 
    return_conf_int=True
)
confint_df = pd.DataFrame(confint)
date_index = pd.date_range(
    start = train_sr.index[-1],
    periods = 3*12,
    freq='MS' # MS = Month Start Frequency
)

predicted_df = pd.DataFrame({'value':predicted_values}, index=date_index)
predicted_df

plt.plot(data_df, label='Actual data')
plt.plot(predicted_df, color='orange', label='Predicted data')
plt.fill_between(date_index, confint_df[0], confint_df[1],color='grey',alpha=.3, label='Confidence Intervals Area')

plt.legend()
plt.show()

In [ ]:
total_sales_last_month = data_df.values[-1][0]
total_sales_next_month = predicted_values[0]

print(f"Last month: {total_sales_last_month}\nNext month: {total_sales_next_month}")

<a id="section-5-2"></a>
## Submission

In [ ]:
# V5 - Score: 1.2276

# pairs['item_cnt_month'] = total_sales_next_month / len(pairs)
# results_df = pairs.drop(['shop_id', 'item_id'], axis=1)
# results_df.to_csv('submission.csv', index=False)
# results_df

In [ ]:
# # V7 - Score: 3.64507

# grouped_data_df = sales \
#     .groupby(['shop_id', 'item_id']) \
#     ['date', 'item_cnt_day'] \
#     .agg({'item_cnt_day':'sum'})
# grouped_data_df.rename(columns={"item_cnt_day": "item_cnt_all"}, inplace=True)
# grouped_data_df = grouped_data_df.reset_index()

# proportion_df = pd.merge(pairs, grouped_data_df, how='left', on=['shop_id', 'item_id']).fillna(1.0)
# prop_sum = proportion_df['item_cnt_all'].sum()

# proportion_df['item_cnt_month'] = total_sales_next_month / prop_sum * proportion_df['item_cnt_all']
# results_df = proportion_df.drop(['shop_id', 'item_id', 'item_cnt_all'], axis=1)
# results_df.to_csv('submission.csv', index=False)
# results_df

In [ ]:
# V21 - Score: 1.21806

grouped_data_df = sales \
    .groupby(['shop_id', 'item_id']) \
    ['date', 'item_cnt_day'] \
    .agg({'item_cnt_day':'sum'})
grouped_data_df.rename(columns={"item_cnt_day": "item_cnt_all"}, inplace=True)
grouped_data_df = grouped_data_df.reset_index()

pairs['item_cnt_month'] = total_sales_next_month / len(pairs) * (len(pairs)/len(grouped_data_df))
results_df = pairs.drop(['shop_id', 'item_id'], axis=1)
results_df.to_csv('submission.csv', index=False)
results_df

**Score**: 1.21806